In [38]:
from datetime import date


def convert_updown_number(text) -> float:
    if text.find('보합') == 0:
        return 0.0
    
    tokens = text.split()
    conv = convert_number(tokens[1])
    
    if tokens[0] == '하락':
        return conv * -1
    else :
        return conv

tx = '''하락
				1,425'''
print(convert_updown_number(tx))

def convert_number(text) -> float:
    negative = 1
    if text[0] == '+':
        text = text[1:]
        negative = 1
    elif text[0] == '-':
        text = text[1:]
        negative = -1
        
    if text.endswith('%'):
        text = text[:-1]
        
    text = text.replace(',', '')
    return float(text) * negative

print(convert_number('12,123,123'))


-1425.0
12123123.0


In [39]:
from typing import Optional, List
from bs4 import BeautifulSoup, Tag
import requests

class DataCompFor:
    def __init__(self, date, end_value, delta, updown_per, volume, comp_buy, for_buy, for_cont, for_per):
        self.date = date
        self.end_value = end_value
        self.delta = delta
        self.updown_per = updown_per
        self.volume = volume
        self.comp_buy = comp_buy
        self.for_buy = for_buy
        self.for_cont = for_cont
        self.for_per = for_per

    def __str__(self):
        return "date: " + self.date + ", end_value: " + self.end_value

def get_stock_data_list(url) -> List[DataCompFor]:
    req = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    dom = BeautifulSoup(req.text, "html.parser")
    sel = dom.select('table.type2 > tr > td.tc')

    sel_c: List[DataCompFor] = []
    for trd in sel:
        tr = trd.parent
        
        try :
            sel_c.append(DataCompFor(
                tr.select_one('td:nth-child(1)').text,
                convert_number(tr.select_one('td:nth-child(2)').text),
                convert_updown_number(tr.select_one('td:nth-child(3)').text.strip()),
                convert_number(tr.select_one('td:nth-child(4)').text.strip()),
                convert_number(tr.select_one('td:nth-child(5)').text),
                convert_number(tr.select_one('td:nth-child(6)').text),
                convert_number(tr.select_one('td:nth-child(7)').text),
                convert_number(tr.select_one('td:nth-child(8)').text),
                convert_number(tr.select_one('td:nth-child(9)').text)
            ))
        except e:
            print('-- parsing error -->' + tr.select_one('td:nth-child(3)').text.strip())
            # print(e)
            
    return sel_c

data = get_stock_data_list(url = 'https://finance.naver.com/item/frgn.naver?code=233740&page=3')

for e in data:
    print(e.__dict__)
    

{'date': '2024.05.30', 'end_value': 9980.0, 'delta': -145.0, 'updown_per': -1.43, 'volume': 17973545.0, 'comp_buy': -980238.0, 'for_buy': -94695.0, 'for_cont': 422594.0, 'for_per': 0.34}
{'date': '2024.05.29', 'end_value': 10125.0, 'delta': -370.0, 'updown_per': -3.53, 'volume': 18219918.0, 'comp_buy': -3469658.0, 'for_buy': -138604.0, 'for_cont': 517289.0, 'for_per': 0.43}
{'date': '2024.05.28', 'end_value': 10495.0, 'delta': -30.0, 'updown_per': -0.29, 'volume': 15739932.0, 'comp_buy': 1430450.0, 'for_buy': -16534.0, 'for_cont': 555893.0, 'for_per': 0.47}
{'date': '2024.05.27', 'end_value': 10525.0, 'delta': 330.0, 'updown_per': 3.24, 'volume': 20613224.0, 'comp_buy': 449751.0, 'for_buy': 269720.0, 'for_cont': 472427.0, 'for_per': 0.41}
{'date': '2024.05.24', 'end_value': 10195.0, 'delta': -195.0, 'updown_per': -1.88, 'volume': 17197420.0, 'comp_buy': -3125331.0, 'for_buy': -157708.0, 'for_cont': 102707.0, 'for_per': 0.09}
{'date': '2024.05.23', 'end_value': 10390.0, 'delta': -70.0, 

In [29]:
def crawl2db(stockCode, page):
    targetUrl = 'https://finance.naver.com/item/frgn.naver?code=%s&page=%s' % (stockCode, page)
    print(targetUrl)
    data = get_stock_data_list(url = targetUrl)

    conn = pymysql.connect(host='horusa', user='root', password='root', db='zio', charset='utf8mb4')
    cur = conn.cursor()

    sql = "INSERT INTO CRWAL_INVESTOR_STOCK_TYPE(TYPE_CODE, DATEON, END_VALUE, DELTA, UPDOWN_PER, VOLUME, COMP_BUY, FOR_BUY, FOR_CONT, FOR_PER) values ('233740', %s, %s, %s, %s, %s, %s, %s, %s, %s)"

    for row in data:
        try:
            print(row.__dict__)
            cur.execute(sql, (row.date, row.end_value, row.delta, row.updown_per, row.volume,  row.comp_buy, row.for_buy, row.for_cont, row.for_per))
        except:
            print('error > duplicated key - {}'.format(row.date))

    conn.commit()
    conn.close()
    
crawl2db('233740', 2)
    

https://finance.naver.com/item/frgn.naver?code=233740&page=2
{'date': '2024.06.28', 'end_value': 10540.0, 'delta': 10.0, 'updown_per': 0.09, 'volume': 11893377.0, 'comp_buy': -717431.0, 'for_buy': -287952.0, 'for_cont': 821251.0, 'for_per': 0.74}
error > duplicated key - 2024.06.28
{'date': '2024.06.27', 'end_value': 10530.0, 'delta': 20.0, 'updown_per': 0.19, 'volume': 11714495.0, 'comp_buy': 132158.0, 'for_buy': -200558.0, 'for_cont': 1209203.0, 'for_per': 1.11}
error > duplicated key - 2024.06.27
{'date': '2024.06.26', 'end_value': 10510.0, 'delta': -95.0, 'updown_per': -0.9, 'volume': 15581559.0, 'comp_buy': -1836772.0, 'for_buy': 57683.0, 'for_cont': 1409761.0, 'for_per': 1.34}
error > duplicated key - 2024.06.26
{'date': '2024.06.25', 'end_value': 10605.0, 'delta': 75.0, 'updown_per': 0.71, 'volume': 14989722.0, 'comp_buy': -587226.0, 'for_buy': 182292.0, 'for_cont': 1352078.0, 'for_per': 1.23}
error > duplicated key - 2024.06.25
{'date': '2024.06.24', 'end_value': 10530.0, 'delt

In [40]:
import time

for i in range(3, 106):
    print("turn #{}".format(i))
    crawl2db('233740', i)
    time.sleep(1)

turn #3
https://finance.naver.com/item/frgn.naver?code=233740&page=3
{'date': '2024.05.30', 'end_value': 9980.0, 'delta': -145.0, 'updown_per': -1.43, 'volume': 17973545.0, 'comp_buy': -980238.0, 'for_buy': -94695.0, 'for_cont': 422594.0, 'for_per': 0.34}
{'date': '2024.05.29', 'end_value': 10125.0, 'delta': -370.0, 'updown_per': -3.53, 'volume': 18219918.0, 'comp_buy': -3469658.0, 'for_buy': -138604.0, 'for_cont': 517289.0, 'for_per': 0.43}
{'date': '2024.05.28', 'end_value': 10495.0, 'delta': -30.0, 'updown_per': -0.29, 'volume': 15739932.0, 'comp_buy': 1430450.0, 'for_buy': -16534.0, 'for_cont': 555893.0, 'for_per': 0.47}
{'date': '2024.05.27', 'end_value': 10525.0, 'delta': 330.0, 'updown_per': 3.24, 'volume': 20613224.0, 'comp_buy': 449751.0, 'for_buy': 269720.0, 'for_cont': 472427.0, 'for_per': 0.41}
{'date': '2024.05.24', 'end_value': 10195.0, 'delta': -195.0, 'updown_per': -1.88, 'volume': 17197420.0, 'comp_buy': -3125331.0, 'for_buy': -157708.0, 'for_cont': 102707.0, 'for_per'

In [18]:
import time

print('Sleep 5 sec')
time.sleep(5)
print('wake up')

Sleep 5 sec
wake up


In [62]:
import re

def check_number(text):
    regex = re.compile(r'((-)?\d{1,3}(,\d{3})*(\.\d+)?)')

    search = regex.search(text)
    if search:
        print('%s is a number!!' % search.group(1))
    else:
        print('%s is not a number!!' % text)


print(check_number('11,893,377'))

11,893,377 is a number!!
None


In [2]:
## mysql ##
import pymysql
conn = pymysql.connect(host='localhost', user='root', password='root', db='zio', charset='utf8mb4')
cur = conn.cursor()

cur.execute("select * from USER_TABLE")

while(True):
    row = cur.fetchone()
    if row == None:
        break
    data1 = row[0]
    data2 = row[1]
    data3 = row[2]
    print("%5s %15s %d"%(data1, data2, data3))

uuid-test-1234            jeff 27
uuid-test-1235         tompson 37
